In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
import locale

import pandas as pd

from src.lexisnexis_parser import docxs_to_df, split_page_from_section, standardize_df
from src.config import PARAM, PATH_DATA_R, PATH_DATA_I

batches = PARAM.lexisnexis.batches
locale.setlocale(locale.LC_ALL, 'nl_NL')

In [ ]:
for batch in batches:
    path_raw = PATH_DATA_R / batch
    df = docxs_to_df(path_raw)
    df = df.drop_duplicates(subset=['title', 'length'], keep='first')
    df.to_pickle(PATH_DATA_I / f'{batch}.pkl')

    # separate section and page
    df = split_page_from_section(df)

    # extract length as integer
    df['length'] = df.length.str.split(' ').str[0].astype('int')

    # count occurrence of paragraphs
    paragraphs = Counter()
    for article in df.body.values:
        for p in article:
            paragraphs[p] += 1

    dupes = {p:paragraphs[p] for p in paragraphs if paragraphs[p] > 1}
    df_dupes = pd.DataFrame.from_dict(dupes, orient='index', columns=['count'])
    df_dupes.to_pickle(PATH_DATA_I / f"{batch}_paragraph_dupes.pkl")

    # remove duplicate (>2) paragraphs 
    dupes = {p:paragraphs[p] for p in paragraphs if paragraphs[p] > 2}
    df['body_'] = df['body'].apply(lambda paragraphs: [p for p in paragraphs if p not in dupes])
    
    # add body as string
    df['body_str'] = df.body_.str.join('\n')

    # convert date strings to dates
    df['load_date'] = pd.to_datetime(df['load_date'])
    df['publication_date'] = lnp.parse_datestring(
        df['publication_date'],
        split_on=',',
        format='%d %B %Y %A',
        )

    df = standardize_df(df, batch)
    df.to_pickle(PATH_DATA_I / f'{batch}_.pkl')